In [ ]:
import numpy as np

# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [ ]:
%cd /data/kimgh/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-NIA/data'

!python create_data.py nia_data_prep --root_path {root_path}

In [7]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data_temp'
sensor = 'lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

/data/kimgh/CenterPoint-custom/CenterPoint-static
no apex
2023-04-06 10:30:28.474565: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 10:30:29.450429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
exist train frames: 549 exist val frames: 549 exist test_nomal frames: 330 exist test_abnormal frames: 0
100%|████████████████████████████████████████| 330/330 [00:02<00:00, 132.50it/s]
0it [00:00, ?it/s]
1
 71%|█████████████████████████████▏           | 391/549 [02:26<01:05,  2.41it/s]

# # Train
---

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!torchrun --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

!python train.py {CONFIG_PATH} --work_dir {work_dir} --validate

# # Validation
---

In [9]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_lidar.pth'
anno_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data_temp/infos_test_normal_filter_True_lidar.pkl'

!CUDA_VISIBLE_DEVICES=1 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --speed_test

/data/kimgh/CenterPoint-custom/CenterPoint-static
No APEX!
no apex
2023-04-06 15:01:54.503827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 15:01:55.936502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
Deformable Convolution not built!
No APEX!
/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py
2023-04-06 15:01:56,858 - INFO - Sensor: lidar
2023-04-06 15:01:56,858 - INFO - Config: /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py
2023-04-06 15:01:56,859 - INFO - Distributed testing: False
2023-04-06 15:01:56,859 - INFO - torch.backends.cudnn.benchmark: False
2023-04-06 15